In [1]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
import datetime as dt
import matplotlib.pyplot as plt
import sys
import time

In [1]:
from pathlib import Path
Path.cwd()

WindowsPath('C:/Users/balob/Documents/GITLAB/RB_BD')

## 1. Отчет по странам с небольшим или отрицательным профитом

In [1]:
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

In [3]:
sql_srt=\
'''
--Calculate total charge and cost by MCC, MNC, usage_type, account_id
select CONVERT(VARCHAR(8), u.usage_date, 3) as 'REP_DATE'
,u.mcc,u.mnc,u.usage_type,uc.account_id
,sum(u.quantity) as quantity,sum(uc.charge) as charge,sum(uc.cost) as cost from [10028_usage2018-04].[dbo].[USAGE] u
inner join [10028_usage2018-04].[dbo].[USAGE_CHARGES] uc
on u.USAGE_ID = uc.USAGE_ID
group by CONVERT(VARCHAR(8), u.usage_date, 3),u.mcc,u.mnc,u.usage_type,uc.account_id
order by u.mcc,CONVERT(VARCHAR(8), u.usage_date, 3)
'''

In [4]:
df = pd.read_sql_query(sql_srt, cnxn,parse_dates={'REP_DATE': '%d/%m/%y'})
df.head()

,REP_DATE,mcc,mnc,usage_type,account_id,quantity,charge,cost
0,2018-04-01,202,5,15,352321.0,4.05000,0.00000,0.00000
1,2018-04-01,202,5,15,352370.0,278.61663,0.00000,0.00000
2,2018-04-01,202,5,21,352321.0,18.00000,0.00000,0.00000
3,2018-04-01,202,5,21,352334.0,26.00000,0.00000,0.00000
4,2018-04-01,202,5,22,352334.0,63.00000,0.63945,0.37044


In [5]:
def f(x):
    data=x[['charge','cost']]
    x['profit']=data['charge']-data['cost']
    if data['cost']!=0:
        x['profit_per']=round((x['profit']/data['cost'])*100,1)
    else:
        x['profit_per']=0
    return x

df1=df.groupby(['mcc','mnc']).agg({'charge': np.sum, 'cost': np.sum}).apply(f,axis=1).copy()

In [6]:
df1[df1['profit']<0]

charge         cost      profit  profit_per
mcc mnc                                                 
208 10      2.37834     34.06715   -31.68881       -93.0
    29    233.39442    274.69781   -41.30339       -15.0
214 9      24.09512    676.18343  -652.08831       -96.4
234 3       0.27545      0.91020    -0.63475       -69.7
240 10      0.07635      0.14915    -0.07280       -48.8
310 260    10.61740    677.87273  -667.25533       -98.4
    680  1465.88595   2125.99344  -660.10749       -31.0
515 2    6012.19448  10894.22103 -4882.02655       -44.8
901 18      0.74520      8.12001    -7.37481       -90.8

In [7]:
df2=df.groupby(['REP_DATE','mcc','mnc']).agg({'charge': np.sum, 'cost': np.sum}).apply(f,axis=1).copy()
df2.head()

,,,charge,cost,profit,profit_per
REP_DATE,mcc,mnc,,,,
2018-02-16,530,1,0.00026,0.00015,0.00011,73.3
2018-02-21,222,10,0.78593,0.45280,0.33313,73.6
2018-02-22,222,10,0.28601,0.16414,0.12187,74.2
2018-02-23,222,10,0.05382,0.02994,0.02388,79.8
2018-02-24,222,10,0.09339,0.05312,0.04027,75.8


In [9]:
df2[(df2.index.get_level_values('mcc')==310) & (df2.index.get_level_values('mnc')==680)].head()

,,,charge,cost,profit,profit_per
REP_DATE,mcc,mnc,,,,
2018-04-01,310,680,78.21434,72.55151,5.66283,7.8
2018-04-02,310,680,42.00338,53.25511,-11.25173,-21.1
2018-04-03,310,680,64.79633,61.03443,3.76190,6.2
2018-04-04,310,680,173.01345,144.49367,28.51978,19.7
2018-04-05,310,680,29.24078,47.55932,-18.31854,-38.5


## 2. Сравнительный анализ цен в TAP и Cost в OCS

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import pyodbc
import mysql.connector #pip install mysql-connector-python-rf
from os.path import join, normpath

In [9]:
# Задать переменные и загрузить вспомогательные данные

data_tariff_allowed = 0.1 # Тариф для data в USD, который рассматривается как discounter
tap_data_tot_charge_threshold = 10 # Полная цена по PLMN, ниже которой не анализируется разница в процентах между тарифами ТАР и OCS
tap_ocs_tariff_diff = 100 # Допустимая разница в процентах между тарифами ТАР и OCS

downloads = 'C:/Users/balob/Downloads'

df_al=pd.read_csv(join(downloads,'DMI.dmi_allowed_list_export_Fri_Feb_15_2019.csv'))

### Получение данных

In [3]:
#Из базы данных BSS выгружаем данные по тарифам

cnxn = mysql.connector.connect(user='noc', password='WcQUzkXiXwoxnFfGnRxb',host='172.18.11.40',database='BSS')

sql_srt=\
'''
SELECT
VisitedNetworkTadig AS TADIG
,CostTariffId AS Price_Plan
,MIN((Cost*1024*1024)/TotalChargedUnits) AS tap_data_min
,AVG((Cost*1024*1024)/TotalChargedUnits) AS tap_data_avg
,STDDEV_SAMP((Cost*1024*1024)/TotalChargedUnits) AS tap_data_std
,MAX((Cost*1024*1024)/TotalChargedUnits) AS tap_data_max
,SUM(Cost) AS tap_data_tot_charge
,SUM(Revenue) AS tap_data_tot_revenue
FROM TAP.GPRS_CALL
WHERE
TreatedTimestampUtc > 20190210000000
AND TotalChargedUnits > 1024
GROUP BY VisitedNetworkTadig,CostTariffId
'''

df_tap_tariff = pd.read_sql_query(sql_srt, cnxn)
cnxn.close()
df_tap_tariff.head()

,TADIG,Price_Plan,tap_data_min,tap_data_avg,tap_data_std,tap_data_max,tap_data_tot_charge,tap_data_tot_revenue
0,AIACW,Partner Cost Plan 2018,0.025735,0.025735,NaN,0.025735,0.047753,0.095378
1,ALBVF,Partner Cost Plan 2018,0.000000,0.002310,0.001473,0.006027,29.300595,58.606340
2,ANDMA,Partner Cost Plan 2018,0.200024,0.200024,NaN,0.200024,3.526519,7.052181
3,ARGTM,Partner Cost Plan 2018,0.000000,0.017668,0.006927,0.047275,506.404190,1012.003865
4,ARM01,Partner Cost Plan 2018,0.004990,0.005046,0.000117,0.005941,67.257209,134.507468


In [4]:
#Из базы данных OCS выгружаем тарифы и объединяем с данными BSS и сохраняем в файл csv

import pandas as pd
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

sql_srt=\
'''
SELECT t.tadig_code AS TADIG,p.roaming_plan_name  AS Price_Plan
,CASE WHEN rpr.currency_id=1 THEN rpr.mo_call_rate WHEN rpr.currency_id=2 THEN rpr.mo_call_rate*1.2 END ocs_moc_rate
,CASE WHEN rpr.currency_id=1 THEN rpr.mt_call_rate WHEN rpr.currency_id=2 THEN rpr.mt_call_rate*1.2 END ocs_mtc_rate
,CASE WHEN rpr.currency_id=1 THEN rpr.mo_sms_rate WHEN rpr.currency_id=2 THEN rpr.mo_sms_rate*1.2 END ocs_mo_sms_rate
,CASE WHEN rpr.currency_id=1 THEN rpr.mt_sms_rate WHEN rpr.currency_id=2 THEN rpr.mt_sms_rate*1.2 END ocs_mt_sms_rate
,CASE WHEN rpr.currency_id=1 THEN rpr.data_rate WHEN rpr.currency_id=2 THEN rpr.data_rate*1.2 END ocs_data_rate
,rpr.is_discounted
FROM ROAMING_PLAN_RULES rpr
LEFT JOIN RDB_TADIG_CODES t
ON rpr.network_id=t.network_id
LEFT JOIN ROAMING_PLANS p
ON rpr.roaming_plan_id=p.roaming_plan_id
WHERE rpr.roaming_plan_id IN (267,268)
AND rpr.end_date IS NULL
AND p.deleted IS NULL
'''

df_ocs_tariff = pd.read_sql_query(sql_srt, cnxn)
cnxn.close()

df = pd.merge(df_tap_tariff,df_ocs_tariff[['TADIG','Price_Plan','ocs_data_rate']],
        how='left',on=['TADIG','Price_Plan'])

df['pct_diff'] = (df['tap_data_avg'] - df['ocs_data_rate'])*100 / df['ocs_data_rate']

#df.to_csv('./DATA/tap_ocs_analysis_180817.csv',index=False)
df.head()

,TADIG,Price_Plan,tap_data_min,tap_data_avg,tap_data_std,tap_data_max,tap_data_tot_charge,tap_data_tot_revenue,ocs_data_rate,pct_diff
0,AIACW,Partner Cost Plan 2018,0.025735,0.025735,NaN,0.025735,0.047753,0.095378,0.0257,0.135226
1,ALBVF,Partner Cost Plan 2018,0.000000,0.002310,0.001473,0.006027,29.300595,58.606340,0.0030,-23.006379
2,ANDMA,Partner Cost Plan 2018,0.200024,0.200024,NaN,0.200024,3.526519,7.052181,0.0500,300.048548
3,ARGTM,Partner Cost Plan 2018,0.000000,0.017668,0.006927,0.047275,506.404190,1012.003865,0.0180,-1.842923
4,ARM01,Partner Cost Plan 2018,0.004990,0.005046,0.000117,0.005941,67.257209,134.507468,0.0050,0.929945


### Анализ данных

In [13]:
#Для анализа можно загрузить уже имеющиеся данные

df = pd.read_csv('./DATA/tap_ocs_analysis_180817.csv')

In [7]:
# Находим тарифы TAP выше заданного допустимого уровня для данного типа тарифа

df[df.tap_data_avg > data_tariff_allowed].sort_values(by=['tap_data_avg'],ascending=False)

,TADIG,Price_Plan,tap_data_min,tap_data_avg,tap_data_std,tap_data_max,tap_data_tot_charge,tap_data_tot_revenue,ocs_data_rate,pct_diff
242,VENMS,Partner Cost Plan 2018,0.193263,0.204001,0.014924,0.337150,63.557946,127.029439,0.20,2.000636
2,ANDMA,Partner Cost Plan 2018,0.200024,0.200024,NaN,0.200024,3.526519,7.052181,0.05,300.048548


In [10]:
# Находим тарифы с наибольшей разницей между данными ТАР и тарифами OCS

df[(df.tap_data_tot_charge > tap_data_tot_charge_threshold)&(df.pct_diff > tap_ocs_tariff_diff)]\
.sort_values(by=['pct_diff'],ascending=False)

,TADIG,Price_Plan,tap_data_min,tap_data_avg,tap_data_std,tap_data_max,tap_data_tot_charge,tap_data_tot_revenue,ocs_data_rate,pct_diff
112,INDA6,Partner Cost Plan 2018,0.00998,0.034935,0.022349,0.099698,15.907983,22.020837,0.01,249.349418


In [70]:
# Анализ данных по Allowed list в DMI и тарифов в OCS

tadig = 'GBRVF'

display(df_ocs_tariff[df_ocs_tariff.TADIG == tadig])
display(df[df.TADIG == tadig])
display(df_al[(df_al['PLMN code'] == tadig) & (df_al['Sponsor'].str.contains('', na=False))])

,TADIG,Price_Plan,ocs_moc_rate,ocs_mtc_rate,ocs_mo_sms_rate,ocs_mt_sms_rate,ocs_data_rate,is_discounted
187,GBRVF,P4 Cost Plan 2018,0.0540,0.0,0.0156,0.0,0.00756,True
603,GBRVF,Partner Cost Plan 2018,0.0439,0.0,0.0062,0.0,0.00300,True


,TADIG,Price_Plan,tap_data_min,tap_data_avg,tap_data_std,tap_data_max,tap_data_tot_charge,tap_data_tot_revenue,ocs_data_rate,pct_diff
86,GBRVF,P4 Cost Plan 2018,0.007128,0.007243,0.00031,0.009504,15.591208,27.431947,0.00756,-4.19221
87,GBRVF,Partner Cost Plan 2018,0.000000,0.003064,0.00091,0.007125,594.488656,1188.754238,0.00300,2.11821


,Sponsor,PLMN code,MO,MT,SMS,Data,MO.1,sec,Data.1,kb
17,Partner_WeConnec,GBRVF,1,1,1,1,1,1,NaN,NaN
140,Partner,GBRVF,1,1,1,1,1,1,NaN,NaN
510,Partner_mondic,GBRVF,1,1,1,1,1,1,NaN,NaN
1318,P4_KeepGo_Test,GBRVF,1,1,1,1,1,1,NaN,NaN
1440,Partner_RedTea,GBRVF,1,1,1,1,1,1,NaN,NaN
1989,Partner_WeStream,GBRVF,1,1,1,1,1,1,NaN,NaN
2169,PA_Telzar_Soft,GBRVF,1,1,1,1,1,1,NaN,NaN
2472,Partner_Dialoq_A,GBRVF,1,1,1,1,1,1,NaN,NaN
2683,Partner_All,GBRVF,1,1,1,1,1,1,NaN,NaN
3105,Partner_RedT_Sof,GBRVF,1,1,1,1,1,1,NaN,NaN


In [12]:
df_al[(df_al['PLMN code'].str.startswith('PRI', na=False)) & (df_al['Sponsor'].str.contains('Partner', na=False))] 

,Sponsor,PLMN code,MO,MT,SMS,Data,MO.1,sec,Data.1,kb


In [40]:
#Анализ данных из BSS по заданному тарифу

cnxn = mysql.connector.connect(user='noc', password='WcQUzkXiXwoxnFfGnRxb',host='172.18.11.40',database='BSS')

sql_srt=\
'''
SELECT
DATE_FORMAT(Search_utcTimeStamp, "%Y-%m-%d %H:%i") AS RED_DATE
,VisitedNetworkTadig
,Cost
,CostTariffId
,TotalChargedUnits
,(Cost*1024*1024)/TotalChargedUnits AS Price
,GprsDest_accessPointNameNI
,InTapFileName
,OutTapFileName
FROM TAP.GPRS_CALL
WHERE
TreatedTimestampUtc > 20190201000000
AND VisitedNetworkTadig = 'MOZVC' AND Cost > 0
AND CostTariffId LIKE 'Partner%'
LIMIT 300
'''

df_tap = pd.read_sql_query(sql_srt, cnxn)
cnxn.close()
df_tap.take([0,1,2,4,5,6,7],axis=1).head()

,RED_DATE,VisitedNetworkTadig,Cost,TotalChargedUnits,Price,GprsDest_accessPointNameNI,InTapFileName
0,2019-02-14 09:03,MOZVC,0.000937,48474,0.020268,globaldata,syniversetap/incoming/CDMOZVCISR9705177
1,2019-02-14 09:18,MOZVC,0.006308,329997,0.020044,globaldata,syniversetap/incoming/CDMOZVCISR9705177
2,2019-02-14 03:46,MOZVC,0.000019,163,0.125387,globaldata,syniversetap/incoming/CDMOZVCISR9705177
3,2019-02-14 11:41,MOZVC,0.001289,67179,0.020123,globaldata,syniversetap/incoming/CDMOZVCISR9705177
4,2019-02-14 05:23,MOZVC,0.000097,4925,0.020749,globaldata,syniversetap/incoming/CDMOZVCISR9705177
5,2019-02-14 11:21,MOZVC,0.005547,290283,0.020036,globaldata,syniversetap/incoming/CDMOZVCISR9705177
6,2019-02-14 15:01,MOZVC,0.000175,8792,0.020922,globaldata,syniversetap/incoming/CDMOZVCISR9705178
7,2019-02-14 13:55,MOZVC,0.007031,367675,0.020051,globaldata,syniversetap/incoming/CDMOZVCISR9705178
8,2019-02-14 19:53,MOZVC,0.001895,98405,0.020191,globaldata,syniversetap/incoming/CDMOZVCISR9705178
9,2019-02-14 16:41,MOZVC,0.000645,33285,0.020307,globaldata,syniversetap/incoming/CDMOZVCISR9705178


# Тестирование

#### Парсинг TAP файлов

In [ ]:
from pandas import DataFrame
from bs4 import BeautifulSoup
tap_file = open(join(downloads,'CPVTM.txt')).read()
#tap_file = open(join(downloads,'TAP.txt')).read()
soup = BeautifulSoup(tap_file, 'xml')
print(soup.find('sender').text,soup.find('localCurrency').text,soup.find('exchangeRate').text)
for callEventDetails in soup.find_all('callEventDetails'):
    print(callEventDetails.imsi.text,
          callEventDetails.charge.text,
          callEventDetails.chargeableUnits.text)